# Patch Extraction
dicom -> Patient -> Study -> DICOMDIR

In [1]:
import pydicom
import os
import glob
from os.path import dirname, join
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import cv2

ModuleNotFoundError: No module named 'pydicom'

In [2]:
# Find DICOMDIR path
for filename in glob.iglob('./dicom/**/DICOMDIR', recursive=True):
    print(filename)
dicomdir_list = glob.glob('./dicom/**/DICOMDIR', recursive=True)
print(f"Number of DICOMDIR: {len(dicomdir_list)}")

./dicom\00006347 KIM SE HO\20170118 Torso PET-CT (G)\DICOMDIR
./dicom\00040607 KIM YOON SOO\20120928 Torso PET-CT (G)\DICOMDIR
./dicom\00107876 BAE SOON DEUK\20140307 Torso PET-CT (G)\DICOMDIR
./dicom\00142351 MIN BOK SOON\20120313 Torso PET-CT (G)\DICOMDIR
./dicom\00167099 NA SOOK HEE\20160310 Torso PET-CT (G)\DICOMDIR
./dicom\00192499 KIM JANG MIN\20170407 Torso PET-CT (G)\DICOMDIR
./dicom\00237841 LEE TAE SOOK\20141024 Torso PET-CT (G)\DICOMDIR
./dicom\00251458 PARK SIN JA\20160328 Torso PET-CT (G)\DICOMDIR
./dicom\00251923 NAM JIN SOON\20160817 Torso PET-CT (G)\DICOMDIR
./dicom\00335345 YOO KI JOO\20160518 Torso PET-CT (G)\DICOMDIR
./dicom\00418386 YOON DONG KOO\20170519 Torso PET-CT (G)\DICOMDIR
./dicom\00450024 CHUN YOO SEONG\20180129 Torso PET-CT (G)\DICOMDIR
./dicom\00459368 CHO JEOM SIM\20170531 Torso PET-CT (G)\DICOMDIR
./dicom\00460582 LEE DEOK SOON\20101129 Torso PET-CT (G)\DICOMDIR
./dicom\00469511 LEE YIN CHEOL\20180803 Torso PET-CT (G)\DICOMDIR
./dicom\00470437 CHO KYOO 

./dicom\01896761 PARK TAE HWA\20140611 Torso PET-CT (G)\DICOMDIR
./dicom\01899726 CHUNG HAE DAE\20140702 Torso PET-CT (G)\DICOMDIR
./dicom\01899816 CHA PIL SEONG\20140701 Torso PET-CT (G)\DICOMDIR
./dicom\01903863 CHA SANG WOOK\20140728 Torso PET-CT (G)\DICOMDIR
./dicom\01917234 CHOI BONG YIL\20170221 Torso PET-CT (G)\DICOMDIR
./dicom\01919957 OH SOON JA\20140930 Torso PET-CT (G)\DICOMDIR
./dicom\01923605 KANG PYEONG JOO\20141029 Torso PET-CT (G)\DICOMDIR
./dicom\01925074 LEE JONG SOOK\20141031 Torso PET-CT (G)\DICOMDIR
./dicom\01928554 KIM YEONG OHK\20141120 Torso PET-CT (G)\DICOMDIR
./dicom\01935706 AHN MI SOOK\20150316 Torso PET-CT (G)\DICOMDIR
./dicom\01941476 HUH DONG WOOK\20150319 Torso PET-CT (G)\DICOMDIR
./dicom\01943199 KANG KYEONG JA\20160205 Torso PET-CT (G)\DICOMDIR
./dicom\01950876 CHAE YI BOK\20170516 Torso PET-CT (G)\DICOMDIR
./dicom\01953860 YANG AE\20170810 Torso PET-CT (G)\DICOMDIR
./dicom\01987688 CHOI SOON KEUM\20180615 Torso PET-CT (G)\DICOMDIR
./dicom\01995226 LEE

In [ ]:
dicom_dir = pydicom.filereader.read_dicomdir(dicomdir_list[35])
base_dir = dirname(dicomdir_list[85])
print(base_dir)

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print(f'Error: Creating directory. ' + directory)

## Extract Slices from DICOM
dicomdir_list[0] -> 146 ~ 184 / total 234 slices on CT
0.62 ~ 0.78

In [ ]:
# Extract All Slices from DICOM
wl = -400
ww = 1600

img_min = wl - ww/2
img_max = wl + ww/2

for dicomdir in dicomdir_list[77:]:
#dicomdir = dicomdir_list[77]

    try:
        ds = pydicom.read_file(dicomdir)    
    except IndexError:
        print(f'Error: dicomdir is empty: ' + dicomdir)
        continue
    print(f'Read {dicomdir}')

    base_dir = dirname(dicomdir)
    base2_dir = dirname(base_dir)

    createFolder(base2_dir + '/PT')
    createFolder(base2_dir + '/CT')

    #pixel_data_CT = list()
    #pixel_data_PET = list()

    # Adjust Window level : -400 and Window width : 1600
    # Min : (Level - Width/2) / Max : (Level + Width/2)

    for record in ds.DirectoryRecordSequence:
        if record.DirectoryRecordType == "IMAGE":
        # Extract the relative path to the DICOM file
            path = os.path.join(base_dir, *record.ReferencedFileID)
            dcm = pydicom.read_file(path)

            # Limit slices for lung
            dcm.InstanceNumber
            # Now get your image data
            if "PT" in dcm.Modality:
                #pixel_data_PET.append(dcm.pixel_array)
                img = dcm.pixel_array
                cv2.imwrite(base2_dir + '/PT/' 
                            + record.ReferencedFileID[-1] + '.png', img)
            else:
                img = dcm.pixel_array
                img_window = np.clip(img+400, 0, 1600)
                img_norm = img_window / 1600 * 255
                #pixel_data_CT.append(img_norm.astype('uint8'))
                cv2.imwrite(base2_dir + '/CT/' 
                            + record.ReferencedFileID[-1] + '.png', img_norm)

In [ ]:
# Copy Slices for lung to new folder
import shutil

max_point = 0.75
min_point = 0.65

#for dicomdir in dicomdir_list:
dicomdir = dicomdir_list[0]

try:
    ds = pydicom.read_file(dicomdir)    
except IndexError:
    print(f'Error: dicomdir is empty: ' + dicomdir)
    continue
print(f'Read {dicomdir}')

base_dir = dirname(dicomdir)
base2_dir = dirname(base_dir)

createFolder(base2_dir + '/PT_lung')
createFolder(base2_dir + '/CT_lung')

num_slices = len(glob.glob(base2_dir + '/CT/*'))

# check slices are in the range
# Find Maximum and minimum and copy all the files through loop
max_slices = np.ceil(max_point * num_slices).astype('uint16') - 1 
min_slices = np.floor(min_point * num_slices).astype('uint16') - 1
print(f"Copy slice #{min_slices} to slice #{max_slices}")
for i in range(min_slices, max_slices):
    try:
        shutil.copy(base2_dir + '/CT/I' + str(i) + '0.png',
               base2_dir + '/CT_lung/I' + str(i) + '0.png',)
    except FileNotFoundError:
        print(f"File Not Found: {base2_dir + '/CT/I' + str(i) + '0.png'}")

    try:
        shutil.copy(base2_dir + '/PT/I' + str(i) + '0.png',
               base2_dir + '/PT_lung/I' + str(i) + '0.png',)
    except FileNotFoundError:
        print(f"File Not Found: {base2_dir + '/CT/I' + str(i) + '0.png'}")

In [ ]:
num_slices = len(glob.glob(base2_dir + '/CT/*'))
print(num_slices)

In [ ]:
# Show slice example
img = pixel_data_CT[155]
cv2.imshow('img', img.astype('uint8'))
while(True):
    k = cv2.waitKey(33)
    if k == -1:
        continue
    else:
        break
cv2.destroyWindow('img')
cv2.imwrite(base_dir + '/CT/I' + str(1560) + '.png', img)

In [ ]:
print(dcm.ImageOrientationPatient)
print(dcm.ImagePositionPatient)
print(dcm.WindowWidth)

In [6]:
dir(ds)

['DirectoryRecordSequence',
 'FileSetConsistencyFlag',
 'FileSetID',
 'OffsetOfTheFirstDirectoryRecordOfTheRootDirectoryEntity',
 'OffsetOfTheLastDirectoryRecordOfTheRootDirectoryEntity',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_character_set',
 '_dataset_slice',
 '_pretty_str',
 '_slice_dataset',
 'add',
 'add_new',
 'clear',
 'convert_pixel_data',
 'data_element',
 'decode',
 'decompress',
 'dir',
 'elements',
 'ensure_file_meta',
 'fix_meta_info',
 'formatted_lines',
 'get',
 'get_item',
 'group_dataset',
 'is_original_encoding',
 'items',
 'iterall',
 'keys',
 'parse_records',
 'pixel_array'

In [8]:
print(record.PatientID)
print(record.PatientName)

00006347
KIM SE HO


In [ ]:
ds.keys()

In [5]:
dicomdir = dicomdir_list[0]
base_dir = dirname(dicomdir)
print(base_dir)

./dicom\00006347 KIM SE HO\20170118 Torso PET-CT (G)


In [7]:
record = ds.DirectoryRecordSequence[0]
dir(record.PatientID)

['DirectoryRecordType',
 'OffsetOfReferencedLowerLevelDirectoryEntity',
 'OffsetOfTheNextDirectoryRecord',
 'PatientBirthDate',
 'PatientID',
 'PatientName',
 'RecordInUseFlag',
 'SpecificCharacterSet',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_character_set',
 '_dataset_slice',
 '_pretty_str',
 '_slice_dataset',
 'add',
 'add_new',
 'clear',
 'convert_pixel_data',
 'data_element',
 'decode',
 'decompress',
 'dir',
 'elements',
 'ensure_file_meta',
 'fix_meta_info',
 'formatted_lines',
 'get',
 'get_item',
 'group_dataset',
 'is_original_encoding',
 'items',
 'iterall',
 'keys',
 'pixel_array',
 